In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00


# 0.MovieLens 데이터를 활용

In [2]:
from torch_geometric.data import download_url, extract_zip
import pandas as pd
import numpy as np
#-- Libreco
# download the dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')

rating_path = '/content/ml-latest-small/ratings.csv'
item_path = "/content/ml-latest-small/movies.csv"

Extracting ./ml-latest-small.zip


In [9]:
df = pd.read_csv(rating_path) #-- Ratings 데이터 읽어오기
print(df.dtypes)
df.head(5) #-- 최상단 5개의 프레임 확인

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# 1. 데이터 확인

### 1-1. 데이터 타입 변경

In [28]:
# df1 = df  #-- 이와 같이 복사하면 Call by value가 아닌 call by reference라 df가 변경되면 df1도 변경되므로 .copy()를 이용하자
df1= df.copy() #-- 복사
# df1 = df1.astype({'rating' : 'object'}) #-- 원하는 칼럼의 데이터 타입을 변경
df1 = df1.astype({'rating' : 'float64', 'userId' : 'float64'}) #-- 원하는 칼럼의 데이터 타입을 여러개 변경
print(df1.dtypes)

userId       float64
movieId        int64
rating       float64
timestamp      int64
dtype: object


### 1-2. 원하는 데이터 출력

In [29]:
print(df1['rating']) #-- userId 데이터만 가져옴

print(df1['rating'].value_counts()) #-- 각각의 값들이 몇 개 들어있는지 확인

0         4.0
1         4.0
2         4.0
3         5.0
4         5.0
         ... 
100831    4.0
100832    5.0
100833    5.0
100834    5.0
100835    3.0
Name: rating, Length: 100836, dtype: float64
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64


# 2. 기초통계량(평균, 중앙값, IQR, 표준편차 등)


In [30]:
df = pd.read_csv(rating_path)
print(df.shape) #-- 행렬 (100836, 4) -> 100836개의 데이터와 4개의 변수로 구성되어 있음
df.head(5)

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### 평균값 구하기(mean)

In [33]:
rating_mean = df['rating'].mean()
print(rating_mean)

3.501556983616962


### 중앙값 구하기(median)

In [35]:
rating_median = df['rating'].median()
print(rating_median)

3.5


### 최빈값 구하기(mode)


In [40]:
rating_mode = df['rating'].mode()
print(rating_mode[0])
# print(df['rating'].value_counts())

4.0


### 분산(var)
분산이 크면 데이터 값들이 평균에서 넓게 퍼져 있다는 것을 의미하고, 분산이 작으면 데이터 값들이 평균에 가까이 모여 있다는 것을 의미한다.

In [41]:
rating_var = df['rating'].var()
print(rating_var)

1.086867214296347


### 표준편차(std)
표준편차가 0에 가깝다는 것은 모든 데이터 값들이 거의 또는 완전히 평균값에 근접해 있다는 것을 의미한다.
데이터 값들이 평균값에서 멀리 떨어져 있을수록 표준편차는 커진다.

In [42]:
rating_std = df['rating'].std()
print(rating_std)

1.042529239060635


### IQR(Q3- Q1)
QR은 데이터의 중간 부분이 얼마나 넓게 퍼져 있는지를 나타냅니다.  

예를 들어, 어떤 시험의 점수가 10, 20, 50, 60, 70, 80, 90점으로 나타났다고 가정해봅시다. 이 경우 중앙값은 60점이고, 하위 50%의 중앙값(Q1)은 20점, 상위 50%의 중앙값(Q3)은 80점일 것입니다. 그러므로 IQR은 80 - 20 = 60점이 됩니다. 이것은 대부분의 학생들이 20점과 80점 사이에 점수를 가지고 있음을 의미하며, 이 범위 밖의 점수는 상대적으로 드물다는 것을 나타냅니다.


In [49]:
Q3 = df['rating'].quantile(.75)
print("Q3 data is : {}".format(Q3))
Q1 = df['rating'].quantile(.25)
print("Q1 data is : {}".format(Q1))
IQR = Q3 - Q1
print("IQR data is : {}".format(IQR))

Q3 data is : 4.0
Q1 data is : 3.0
IQR data is : 1.0


### 최대, 최소

In [55]:
rating_max = df['rating'].max()
print("rating_max value is {}".format(rating_max))
rating_min = df['rating'].min()
print("rating_min value is {}".format(rating_min))
rating_range = rating_max - rating_min
print("rating_range is {}".format(rating_range))

rating_max value is 5.0
rating_min value is 0.5
rating_range is 4.5


### 왜도(skew)

In [57]:
rating_skew = df['rating'].skew()
print(rating_skew)

-0.6371993892529214


### 첨도(kurt)

In [58]:
rating_kurt = df['rating'].kurt()
print(rating_kurt)

0.12331407123922622


# 3. 그룹화하여 데이터 전처리, 데이터 인덱싱, 필터링, 정렬

In [70]:
import seaborn as sns
df = sns.load_dataset('iris')
print(df.head(5)) #-- species 재정렬

df.groupby('species').mean()

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


### 데이터 인덱싱

In [75]:
df = pd.read_csv(rating_path)
print(df.shape) #-- 행렬 (100836, 4) -> 100836개의 데이터와 4개의 변수로 구성되어 있음
df.head(5)
df.loc[3, 'rating'] #-- 3번째가 아닌 인덱스번호 3을 가져옴

(100836, 4)


5.0

In [78]:
df.loc[:, 'rating'] #-- rating만 가져옴
df.loc[0:3, ['rating', 'userId']] #-- 특정 인덱스구간의 원하는 데이터만 추출

,rating,userId
0,4.0,1
1,4.0,1
2,4.0,1
3,5.0,1


In [80]:
df.head(5)#-- 앞에서 5개
df.tail(5)#-- 뒤에서 5개

,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


### 열 추가/제거

In [89]:
df_rating = df.rating
# df_rating = df['rating']
df_rating.head(5)

#-- 여러개 가져오기
df1 = df[['rating', 'userId']]
df1.head(3)

,rating,userId
0,4.0,1
1,4.0,1
2,4.0,1


In [96]:
df_drop = df.copy()
df_drop = df_drop.drop(columns=['timestamp', 'movieId'])
df_drop.head(5)

,userId,rating
0,1,4.0
1,1,4.0
2,1,4.0
3,1,5.0
4,1,5.0


In [98]:
#-- 열 추가
df_new = df.copy()
df_new['new'] = df['rating'].mean()
df_new.head(3)

,userId,movieId,rating,timestamp,new
0,1,1,4.0,964982703,3.501557
1,1,3,4.0,964981247,3.501557
2,1,6,4.0,964982224,3.501557


### 데이터 필터링

In [109]:
ratings = (df["rating"] >=3) #-- 평점이 3이상만 필터링
useID = (df["userId"] <=5) #-- Id가 5이히만 필터링
print(len(df[ratings]))
print(len(df[useID]))

df[ratings & useID] #-- 2개 조건을 필터링

81763
560


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
555,5,594,5.0,847435238
556,5,595,5.0,847434832
557,5,596,5.0,847435292
558,5,597,3.0,847434962


# 4. 데이터 정렬

### 내림차순 정렬

In [6]:
df = pd.read_csv(rating_path)
df.sort_values('rating', ascending = False).head(5)

,userId,movieId,rating,timestamp
34031,232,3147,5.0,1218164903
50717,328,1080,5.0,1494210450
18166,115,487,5.0,944951378
18165,115,480,5.0,944953733
76157,479,3308,5.0,1039362558


### 오름차순 정렬

In [7]:
df = pd.read_csv(rating_path)
df.sort_values('rating', ascending = True).head(5)

,userId,movieId,rating,timestamp
3752,22,53519,0.5,1268727137
60861,393,5445,0.5,1430506636
47025,307,2017,0.5,1186173639
22446,153,1198,0.5,1525548264
60865,393,5902,0.5,1430507509


# 5. 데이터 변경

In [16]:
df = pd.read_csv(rating_path)

#-- rating의 값이 3이하인 값은 0으로 만들고 나머지는 그냥 유지
df['rating'] = np.where(df['rating'] >= 3, df['rating'],0)
#-- rating 프레임의 데이터 중 조건식이 true이면 1번째, false이면 2번째 값 대입

df.sort_values('rating', ascending = True).head(5)

,userId,movieId,rating,timestamp
47138,307,2907,0.0,1186172679
44510,297,1037,0.0,900876328
17071,108,5502,0.0,1042839211
72623,469,1301,0.0,965335350
90251,586,176371,0.0,1529899177


# 데이터 결측치, 이상치, 중복치 제거

In [23]:
#-- 타이타닉 데이터 불러오기
import seaborn as sns
df = sns.load_dataset('titanic')
print(df.shape) #-- 15개의 변수와 891개의 변수
print(df.dtypes)
df.head(5)

#-- 종속 변수 Y : 생존 여부(servied)
#-- 독립 변수 X : 성별, 나이등의 탑승자 정보

(891, 15)
survived          int64
pclass            int64
sex              object
age             float64
sibsp             int64
parch             int64
fare            float64
embarked         object
class          category
who              object
adult_male         bool
deck           category
embark_town      object
alive            object
alone              bool
dtype: object


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### 결측치 확인(isnull())

In [30]:
print(df.isnull().sum()) #-- 결측치 확인
print(df.dropna(axis=0).shape) #-- 행 기준 제거 --> 데이터를 삭제함
print(df.dropna(axis=1).shape) #-- 열 기준 제거 --> 변수를 삭제함

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64
(182, 15)
(891, 11)
